In [1]:
!python -m pip install --upgrade langchain deeplake openai tiktoken

In [2]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [4]:
import os
from langchain.document_loaders import TextLoader

root_dir = "C:\\Users\\Zhang\\IdeaProjects\\PLCParser_Java\\src\\main\\java\\etherip"
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1575, which is longer than the specified 1000
Created a chunk of size 1340, which is longer than the specified 1000
Created a chunk of size 1413, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 2058, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1154, which is longer than the specified 1000
Created a chunk of size 2180, which is longer than the specified 1000
Created a chunk of size 1002, which is longer than the specified 1000
Created a chunk of size 1308, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1401, which is longer than the specified 1000
Created a chunk of size 1401, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of s

In [6]:
username = "bobbyztz"  # replace with your username from app.activeloop.ai
db = DeepLake(
    dataset_path=f"hub://{username}/codeReader",
    embedding_function=embeddings,
)
db.add_documents(texts)

Deep Lake Dataset in hub://bobbyztz/codeReader already exists, loading from the storage


-

Dataset(path='hub://bobbyztz/codeReader', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
 embedding  embedding  (807, 1536)  float32   None   
    id        text      (807, 1)      str     None   
 metadata     json      (807, 1)      str     None   
   text       text      (807, 1)      str     None   


['889214d9-0ed2-11ee-b60b-4851c55e5357',
 '889214da-0ed2-11ee-98a7-4851c55e5357',
 '889214db-0ed2-11ee-b082-4851c55e5357',
 '889214dc-0ed2-11ee-b8f9-4851c55e5357',
 '889214dd-0ed2-11ee-9aea-4851c55e5357',
 '889214de-0ed2-11ee-8297-4851c55e5357',
 '889214df-0ed2-11ee-9092-4851c55e5357',
 '889214e0-0ed2-11ee-bdca-4851c55e5357',
 '889214e1-0ed2-11ee-a0b8-4851c55e5357',
 '889214e2-0ed2-11ee-9f19-4851c55e5357',
 '889214e3-0ed2-11ee-99ea-4851c55e5357',
 '889214e4-0ed2-11ee-bac4-4851c55e5357',
 '889214e5-0ed2-11ee-b229-4851c55e5357',
 '889214e6-0ed2-11ee-b51c-4851c55e5357',
 '889214e7-0ed2-11ee-9369-4851c55e5357',
 '889214e8-0ed2-11ee-bdf0-4851c55e5357',
 '889214e9-0ed2-11ee-9e80-4851c55e5357',
 '889214ea-0ed2-11ee-96ea-4851c55e5357',
 '889214eb-0ed2-11ee-a556-4851c55e5357',
 '889214ec-0ed2-11ee-b910-4851c55e5357',
 '889214ed-0ed2-11ee-ba1e-4851c55e5357',
 '889214ee-0ed2-11ee-8dfa-4851c55e5357',
 '889214ef-0ed2-11ee-a04c-4851c55e5357',
 '889214f0-0ed2-11ee-88e9-4851c55e5357',
 '889214f1-0ed2-

In [7]:
db = DeepLake(
    dataset_path="hub://bobbyztz/codeReader",
    read_only=True,
    embedding_function=embeddings,
)

Deep Lake Dataset in hub://bobbyztz/codeReader already exists, loading from the storage


In [8]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo-16k-0613")  # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [20]:
questions = [
    #"Have you found information about message router and connection manager? As I know they both should be objects in CIP. Please return any information you found such as fields or relationships.",
    #"A lot of classes inherit the Protocol class or ProtocolAdapter class, what are the relationships between these classes?Just like I can pass a SendRRDataProtocol object as a parameter into the constructor of Encapsulation class. Please draw a diagram to show all these relationships by using any proper symbol, and explain them to me.",
    "as you can see In the Main class, I check if the packet has TCP or UDP part and I dig inside Encapsulation part. Then inside Encapsulation class, I have a series of if-else statement to check the command_code and decide which direction to go, like dig into registerSessionProtocol. Can you finish this parsing tree for me in a similar way?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: as you can see In the Main class, I check if the packet has TCP or UDP part and I dig inside Encapsulation part. Then inside Encapsulation class, I have a series of if-else statement to check the command_code and decide which direction to go, like dig into registerSessionProtocol. Can you finish this parsing tree for me in a similar way? 

**Answer**: Unfortunately, I cannot complete the parsing tree for you without further information. The provided code does not contain the complete logic for handling different command codes within the Encapsulation class. Additionally, the code does not provide any information about the structure and logic of the RegisterSessionProtocol class or other potential classes that may be involved in the parsing process.

To complete the parsing tree, you would need to provide the missing code and specify the expected behavior for each command code. Alternatively, if you have access to the complete source code, you can provide the relevant s